In [9]:
import geopandas as gpd

In [1]:
def tile_generator(self, pixel_locations, batch_size, flatten=False, canopy=False):
    ### this is a keras compatible data generator which generates data and labels on the fly 
    ### from a set of pixel locations, a list of image datasets, and a label dataset
    bad_tiles = 0
    tile_size = self.tile_length
    i = 0
    lc_proj = Proj(self.lc_label.crs)
    canopy_proj = Proj(self.canopy_label.crs)
    l8_proj = Proj(self.l8_dict['028012'][0].crs)
    # assuming all images have the same num of bands
    band_count = self.l8_dict['028012'][0].count - 1
    time_steps = len(self.l8_dict['028012'])
    class_count = self.class_count
    buffer = math.floor(tile_size / 2)
    while True:
        lc_batch = np.zeros((batch_size, class_count))
        canopy_batch = np.zeros((batch_size, 1))
        image_batch = np.zeros((batch_size, time_steps, tile_size, tile_size, band_count))
        rnn_image_batch = np.zeros((batch_size, time_steps, band_count))
        b = 0
        while b < batch_size:
            # if we're at the end  of the data just restart
            if i >= len(pixel_locations):
                i=0
            r, c = pixel_locations[i][0]
            tile_num = pixel_locations[i][1]
            i += 1
            tiles_to_read = self.l8_dict[tile_num]
            tiles_read = util.read_windows(tiles_to_read, c ,r, buffer, tile_size)
            rnn_tiles_read = util.read_windows(tiles_to_read, c ,r, 0, 1)
            reshaped_tiles = []
            rnn_reshaped_tiles = []
            band_avg = [  345.72448081,   352.93755735,   319.34257128,   899.39728239,
         649.46125258,   370.53562465, -1084.8218946 ]
            band_std = [ 661.75737932,  363.32761072,  425.28671553,  959.63442896,
        838.86193201,  501.96992987, 3562.42995552]
            for index in range(len(tiles_read)):
                rnn_tile = rnn_tiles_read[index][0:7]
                tile = tiles_read[index][0:7]
                reshaped_tile = reshape_as_image(tile).astype(np.float64)
                rnn_reshaped_tile = reshape_as_image(rnn_tile).astype(np.float64)
                rnn_reshaped_tile = np.divide(np.subtract(rnn_reshaped_tile,band_avg),band_std)                 
                reshaped_tile= np.divide(np.subtract(reshaped_tile, band_avg),band_std)
                reshaped_tiles.append(reshaped_tile)
                rnn_reshaped_tiles.append(rnn_reshaped_tile)
            ### get label data
            # find gps of that pixel within the image
            (x, y) = self.l8_dict[tile_num][0].xy(r, c) 
            # convert the point we're sampling from to the same projection as the label dataset if necessary
            lc_x,lc_y = x,y
            canopy_x, canopy_y = x,y
            # reference gps in label_image
            lc_row, lc_col = self.lc_label.index(lc_x,lc_y)
            lc_data = self.lc_label.read(1, window=Window(lc_col, lc_row, 1, 1))
            canopy_row, canopy_col = self.canopy_label.index(canopy_x,canopy_y)
            canopy_data = self.canopy_label.read(1, window=Window(canopy_col, canopy_row, 1, 1))
            lc_label = self.one_hot_encode(lc_data, 1, class_count)
            lc_batch[b] = lc_label.reshape(class_count)
            canopy_batch[b] = canopy_data.reshape(1) / 100
            rnn_total_tile = np.array((*rnn_reshaped_tiles,))
            rnn_image_batch[b] = rnn_total_tile.reshape((len(tiles_read),7))
            total_tile = np.array((*reshaped_tiles,))
            image_batch[b] = total_tile
            b += 1
            if canopy:
                yield ({"rnn_input":rnn_image_batch, "tile_input":image_batch}, {'landcover': lc_batch, 'canopy': canopy_batch})
            else: 
                yield (image_batch, lc_batch)

In [8]:
gpd.read_file('train_buffered_points.shp')

,canopy,col,label,row,tile_name,x,y,geometry
0,0.00,730,0,4250,028012,1656330.0,1387290.0,POLYGON ((-77.79800241884466 34.11802936166239...
1,0.00,146,0,520,029011,1788810.0,1649190.0,POLYGON ((-75.77941114994056 36.17332863020923...
2,0.00,3413,0,1659,028012,1736820.0,1465020.0,"POLYGON ((-76.766124934262 34.65891474668916, ..."
3,0.00,4294,0,1323,028012,1763250.0,1475100.0,POLYGON ((-76.45891224088588 34.69929438010407...
4,0.00,565,0,3510,029011,1801380.0,1559490.0,POLYGON ((-75.85329201792212 35.36650268777395...
5,0.00,3408,0,397,028012,1736670.0,1502880.0,"POLYGON ((-76.6836822568864 34.99069012167243,..."
6,0.00,1754,0,2916,029011,1837050.0,1577310.0,POLYGON ((-75.42408362455453 35.45431213972186...
7,0.00,966,0,1353,029011,1813410.0,1624200.0,POLYGON ((-75.56958781383901 35.90863525485503...
8,0.00,9,0,4016,029011,1784700.0,1544310.0,POLYGON ((-76.06947330202092 35.26506678291997...
9,0.00,4049,0,4137,028011,1755900.0,1540680.0,"POLYGON ((-76.39021819376609 35.2864805528474,..."


In [14]:
def tile_generator(self, pixel_info, batch_size, flatten=False, canopy=False):
    ### this is a keras compatible data generator which generates data and labels on the fly 
    ### from a set of pixel locations, a list of image datasets, and a label dataset
    bad_tiles = 0
    tile_size = self.tile_length
    i = 0
    # assuming all images have the same num of bands
    band_count = self.l8_dict['028012'][0].count - 1
    time_steps = len(self.l8_dict['028012'])
    class_count = self.class_count
    buffer = math.floor(tile_size / 2)
    while True:
        lc_batch = np.zeros((batch_size, class_count))
        canopy_batch = np.zeros((batch_size, 1))
        image_batch = np.zeros((batch_size, time_steps, tile_size, tile_size, band_count))
        rnn_image_batch = np.zeros((batch_size, time_steps, band_count))
        b = 0
        while b < batch_size:
            # if we're at the end  of the data just restart
            if i >= len(pixel_locations):
                i=0
            r, c = pixel_info["row"][i], pixel_info["col"][i] ####
            tile_num = pixel_info["tile_name"][i] #####
            i += 1
            tiles_to_read = self.l8_dict[tile_num]
            tiles_read = util.read_windows(tiles_to_read, c ,r, buffer, tile_size)
            rnn_tiles_read = util.read_windows(tiles_to_read, c ,r, 0, 1)
            reshaped_tiles = []
            rnn_reshaped_tiles = []
            band_avg = [  345.72448081,   352.93755735,   319.34257128,   899.39728239,
         649.46125258,   370.53562465, -1084.8218946 ]
            band_std = [ 661.75737932,  363.32761072,  425.28671553,  959.63442896,
        838.86193201,  501.96992987, 3562.42995552]
            for index in range(len(tiles_read)):
                rnn_tile = rnn_tiles_read[index][0:7]
                tile = tiles_read[index][0:7]
                reshaped_tile = reshape_as_image(tile).astype(np.float64)
                rnn_reshaped_tile = reshape_as_image(rnn_tile).astype(np.float64)
                rnn_reshaped_tile = np.divide(np.subtract(rnn_reshaped_tile,band_avg),band_std)                 
                reshaped_tile= np.divide(np.subtract(reshaped_tile, band_avg),band_std)
                reshaped_tiles.append(reshaped_tile)
                rnn_reshaped_tiles.append(rnn_reshaped_tile)
            lc_data = pixel_info["label"][i]
            canopy_data = pixel_info["label"][i]
            lc_label = self.one_hot_encode(lc_data, 1, class_count)
            lc_batch[b] = lc_label.reshape(class_count)
            canopy_batch[b] = canopy_data.reshape(1) / 100
            rnn_total_tile = np.array((*rnn_reshaped_tiles,))
            rnn_image_batch[b] = rnn_total_tile.reshape((len(tiles_read),7))
            total_tile = np.array((*reshaped_tiles,))
            image_batch[b] = total_tile
            b += 1
            if canopy:
                yield ({"rnn_input":rnn_image_batch, "tile_input":image_batch}, {'landcover': lc_batch, 'canopy': canopy_batch})
            else: 
                yield (image_batch, lc_batch)

In [10]:
train = gpd.read_file('train_buffered_points.shp')

In [13]:
train["row"]

0       4250
1        520
2       1659
3       1323
4       3510
5        397
6       2916
7       1353
8       4016
9       4137
10      3698
11      1432
12      2052
13      2447
14      4647
15      3109
16      2706
17      4737
18      1225
19      3452
20      3745
21      3993
22      3133
23      4539
24      3752
25      4722
26      2778
27      3814
28      3788
29      1986
        ... 
7225    1523
7226    2705
7227    3364
7228     115
7229    2701
7230    2840
7231    3326
7232    1834
7233     801
7234     927
7235    2419
7236    2825
7237    4199
7238    1306
7239    2862
7240    4590
7241    3724
7242    3953
7243    4898
7244    3798
7245    3701
7246    2571
7247    2541
7248    1082
7249    2695
7250    3773
7251    1763
7252    1408
7253     187
7254    4849
Name: row, Length: 7255, dtype: int64